https://www.youtube.com/watch?v=aBVGKoNZQUw

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
# from inference import get_model
import supervision as sv
import torch
import torchvision
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

print('PyTorch version:', torch.__version__)
print('PyTorch Cuda eligibility:', torch.version.cuda)
print('Torchvision version:', torchvision.__version__)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device is:', device, '\n')
print(os.getcwd())

PyTorch version: 2.5.0
PyTorch Cuda eligibility: 12.1
Torchvision version: 0.20.0
Device is: cuda 

C:\Users\ngyin\Documents\NUS Stackables\SP 1\Practice Module (landmark recognition)\data


In [10]:
model = YOLO("runs/detect/train3/weights/best.pt")
SOURCE_VIDEO_PATH = "sample/online_video_1.mp4"
TARGET_VIDEO_PATH = "sample/online_video_1_result_2.mp4"

In [55]:
box_annotator = sv.BoxAnnotator()
dot_annotator = sv.DotAnnotator(
    radius=15,
    outline_thickness=5,
    outline_color=sv.Color.from_hex('#FFFFFF')
)

rich_label_annotator = sv.RichLabelAnnotator(
    font_path="<TTF_FONT_PATH>",
    font_size=35,
    text_position=sv.Position.TOP_CENTER,
    text_padding=12,
    border_radius=15,
    smart_position=True
)

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
print(video_info)
video_sink = sv.VideoSink(TARGET_VIDEO_PATH, video_info=video_info)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

frame_skip = 40
frame_count = 0

with video_sink:
    for frame in tqdm(frame_generator, total=video_info.total_frames):
        if frame_count % frame_skip == 0:            
            results = model(frame)[0]
            # print(results.boxes)
            # plt.imshow(results.orig_img)
            detections = sv.Detections.from_ultralytics(results)
            detections = detections.with_nms(threshold=0.7, class_agnostic=False)

            # To pick the highest confidence per class. Only 1 detection per frame.
            # Convert detections into a structured dictionary for processing.
            detections_by_class = {}
            for i in range(len(detections.class_id)):
                class_id = detections.class_id[i]
                if class_id not in detections_by_class or detections.confidence[i] > detections_by_class[class_id]['confidence']:
                    detections_by_class[class_id] = {
                        'xyxy': detections.xyxy[i],
                        'confidence': detections.confidence[i],
                        'class_id': class_id,
                        'class_name': detections.data['class_name'][i]
                    }
            
            # Create new filtered detections
            filtered_xyxy = np.array([det['xyxy'] for det in detections_by_class.values()])
            filtered_confidence = np.array([det['confidence'] for det in detections_by_class.values()])
            filtered_class_id = np.array([det['class_id'] for det in detections_by_class.values()])
            filtered_class_name = np.array([det['class_name'] for det in detections_by_class.values()])
            
            # Construct the filtered detections object
            filtered_detections = sv.Detections(
                xyxy=filtered_xyxy,
                confidence=filtered_confidence,
                class_id=filtered_class_id,
                data={'class_name': filtered_class_name}
            )
            
            # labels = [
            #     f"{class_name}  {confidence:.2f}"
            #     for class_name, confidence
            #     in zip(detections['class_name'], detections.confidence)
            # ]

            labels = [
                f"{class_name}  {confidence:.2f}"
                for class_name, confidence
                in zip(filtered_detections.data['class_name'], filtered_detections.confidence)
            ]            
            
            annotated_frame = frame.copy()
            annotated_frame = dot_annotator.annotate(annotated_frame, filtered_detections)
            annotated_frame = dot_annotator.annotate(annotated_frame, filtered_detections)
            annotated_frame = rich_label_annotator.annotate(annotated_frame, filtered_detections, labels=labels)
            video_sink.write_frame(annotated_frame)
        frame += 1

# sv.plot_image(annotated_frame)



Font path '<TTF_FONT_PATH>' not found. Using PIL's default font.
VideoInfo(width=2562, height=1440, fps=25, total_frames=207)


  0%|                                                                                          | 0/207 [00:00<?, ?it/s]


0: 384x640 1 art_sci_museum, 1 helix_bridge, 1 lv_marina_bay, 1 mbs, 1 mbs_hotel, 1 sg_river, 53.4ms
Speed: 6.2ms preprocess, 53.4ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)


  0%|▍                                                                                 | 1/207 [00:00<01:13,  2.80it/s]


0: 384x640 1 art_sci_museum, 1 helix_bridge, 1 mbs, 1 mbs_hotel, 1 sg_river, 22.2ms
Speed: 2.8ms preprocess, 22.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)


...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 205/207 [00:33<00:00,  5.63it/s]


0: 384x640 1 apple_marina_bay, 1 art_sci_museum, 1 helix_bridge, 1 lv_marina_bay, 1 mbs, 1 mbs_hotel, 1 sg_river, 35.7ms
Speed: 4.3ms preprocess, 35.7ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)


100%|███████████████████████████████████████████████████████████████████████████████▌| 206/207 [00:33<00:00,  5.53it/s]


0: 384x640 1 apple_marina_bay, 1 art_sci_museum, 1 helix_bridge, 1 lv_marina_bay, 1 mbs, 1 mbs_hotel, 1 sg_river, 36.6ms
Speed: 4.2ms preprocess, 36.6ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)


100%|████████████████████████████████████████████████████████████████████████████████| 207/207 [00:33<00:00,  6.18it/s]


In [54]:
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
test_f = next(frame_generator)
# sv.plot_image(test_f)
detections = sv.Detections.from_ultralytics(results).with_nms(threshold=0.7, class_agnostic=False)
detections

Detections(xyxy=array([[     441.41,      615.53,      1143.3,      977.48],
       [     1.5329,      856.95,      386.62,      1085.7],
       [     72.525,      1017.3,      2560.2,        1440],
       [     612.22,      1011.4,      791.76,      1087.7],
       [     363.24,      904.92,      1704.9,        1050],
       [       1782,      961.34,      2345.7,      1009.9],
       [     1356.7,      967.16,      1452.5,      1049.7]], dtype=float32), mask=None, confidence=array([    0.92882,      0.8847,      0.8781,     0.78996,     0.77863,     0.61992,     0.54826], dtype=float32), class_id=array([13,  3, 18, 11, 12,  9,  2]), tracker_id=None, data={'class_name': array(['mbs_hotel', 'art_sci_museum', 'sg_river', 'lv_marina_bay', 'mbs', 'helix_bridge', 'apple_marina_bay'], dtype='<U16')}, metadata={})